In [61]:
from google.cloud import bigquery
import tensorflow as tf

import pandas as pd

import json
import os
from datetime import date,datetime,timedelta

import functions_framework

from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest

In [62]:
# @functions_framework.http
# def predict_incident_severity_by_tf(request):

In [63]:
# load_model_option=os.environ.get('load_model_option', '1')
load_model_option='1'

# predict_from_date=os.environ.get('predict_from_date', '')
predict_from_date='2000-01-01'
predict_from_date=''

if load_model_option=='1':
  PATH_FOLDER_ARTIFACTS="model"  
elif load_model_option=='2':
  PATH_FOLDER_ARTIFACTS="gs://tf1-incident-pongthorn/model"
else:
  raise Exception("Allow you to set 1 for local and 2 for google storage")

print(f"Load data from {PATH_FOLDER_ARTIFACTS}")

Load data from model


In [64]:
table_id = "pongthorn.SMartML.new_incident"
predictResult_table_id="pongthorn.SMartML.new_result_prediction_incident"


unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']

In [65]:
mapping_file="incident_sevirity_to_class.json"
# with open(mapping_file, 'r') as json_file:
#      map_sevirity_to_class= json.load(json_file)
# print(map_sevirity_to_class)

map_sevirity_to_class={'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}
print(map_sevirity_to_class)

{'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [66]:
# Get today's date
prediction_datetime=datetime.now()

today = date.today()

# Yesterday date
if predict_from_date=='':
 yesterday = today - timedelta(days = 1)
 str_yesterday=yesterday.strftime('%Y-%m-%d')
else:
 str_yesterday=predict_from_date

str_today=today.strftime('%Y-%m-%d')

print(f"Get data between {str_yesterday} to {str_today} to predict sevirity level")

Get data between 2023-03-29 to 2023-03-30 to predict sevirity level


In [67]:
def load_data_bq(sql:str):
 client_bq = bigquery.Client()
 query_result=client_bq.query(sql)
 df=query_result.to_dataframe()
 return df

In [68]:
sql=f"""
SELECT *  FROM `{table_id}` 
WHERE DATE(imported_at) >= '{str_yesterday}' and DATE(imported_at) < '{str_today}' 

"""
#LIMIT 2
dfNewData=load_data_bq(sql)
dfNewData=dfNewData.drop_duplicates(subset=['id'],keep='first')

dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)


print(dfNewData.info())
print(dfNewData)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         1 non-null      Int64         
 1   severity_id                1 non-null      Int64         
 2   severity                   1 non-null      int64         
 3   severity_name              1 non-null      object        
 4   sla                        1 non-null      object        
 5   product_type               1 non-null      object        
 6   brand                      1 non-null      object        
 7   service_type               1 non-null      object        
 8   incident_type              1 non-null      object        
 9   open_to_close_hour         1 non-null      float64       
 10  response_to_resolved_hour  1 non-null      float64       
 11  imported_at                1 non-null      datetime64[ns]
dtypes: Int64(2),

In [69]:
try:
    model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
    print(f"Load from {PATH_FOLDER_ARTIFACTS}")
    print(model.summary())
except Exception as error:
  print(str(error))
  raise error


Load from model
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sla (InputLayer)               [(None, 1)]          0           []                               
                                                                                                  
 product_type (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 brand (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 service_type (InputLayer)      [(None, 1)]          0           []                               
                                                                            

In [56]:
pdPrediction=pd.DataFrame(columns=['_id','predict_severity','prob_severity'])

for  row_dict in dfNewData.to_dict(orient="records"):
      incident_id=row_dict['id']
      print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
      for key_removed in unUsedColtoPredict:
       row_dict.pop(key_removed)
      # print(row_dict)  

      input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}


      predictionResult = model.predict(input_dict)
      result_str=','.join([ str(prob) for prob in predictionResult[0]])  
      print(result_str)   

      prob = tf.nn.softmax(predictionResult)
      prob_pct=(100 * prob)  
      _class = tf.argmax(predictionResult,-1).numpy()[0]
      
      dictPrediction={'_id':incident_id, 'predict_severity':_class,'prob_severity':result_str} 
      pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([dictPrediction])] )

      print(f"{prob_pct} %   as {_class}")     
      print("======================================================================================")
            
dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
dfPredictData=dfPredictData.drop(columns=['_id'])
dfPredictData['predict_severity']=dfPredictData['predict_severity'].astype('int')
dfPredictData=dfPredictData[['id','prob_severity','predict_severity','severity']]
dfPredictData['prediction_item_date']= datetime.strptime(str(yesterday), '%Y-%m-%d')
dfPredictData['prediction_datetime']=prediction_datetime

2570 - 0(Cosmatic)
1/1 [==============================] - 1s 622ms/step
0.30158505,0.6914548,0.0069139185,4.6226673e-05
[[25.244595 37.280926 18.80158  18.6729  ]] %   as 1


In [57]:
print(dfPredictData.info())
print(dfPredictData)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    1 non-null      object        
 1   prob_severity         1 non-null      object        
 2   predict_severity      1 non-null      int32         
 3   severity              1 non-null      int64         
 4   prediction_item_date  1 non-null      datetime64[ns]
 5   prediction_datetime   1 non-null      datetime64[ns]
dtypes: datetime64[ns](2), int32(1), int64(1), object(2)
memory usage: 52.0+ bytes
None
     id                                    prob_severity  predict_severity  \
0  2570  0.30158505,0.6914548,0.0069139185,4.6226673e-05                 1   

   severity prediction_item_date        prediction_datetime  
0         0           2023-03-29 2023-03-30 22:03:37.861321  


In [58]:
#https://cloud.google.com/bigquery/docs/samples/bigquery-create-table#bigquery_create_table-python

try:
    client = bigquery.Client()
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("prob_severity", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("predict_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity", "INTEGER", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_item_date", "DATETIME", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") 
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Predict Result Table pongthorn.SMartML.new_result_prediction_incident already exists.


In [59]:
def loadDataFrameToBQ():
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            dfPredictData, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(dfPredictData), "Imported igquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  1 Imported igquery successfully


In [ ]:
# return 'ok'

In [ ]:
# if __name__ == "__main__":
#  result=predict_incident_severity_by_tf(None)
#  print(result)
